In [1]:
from pathlib import Path
import os, json, torch
from datasets import Dataset
from transformers import (
    AutoTokenizer, AutoModelForCausalLM,
    BitsAndBytesConfig, TrainingArguments,
    DataCollatorForLanguageModeling, Trainer
)
from peft import get_peft_model, LoraConfig

# ---------- Paths ----------
DATA_PATH  = Path(r"C:\Users\yifan\yifanbot\data\tone.jsonl")
OUTPUT_DIR = Path("./lora-Qwen2.5-7B")      # where to save adapter

# ---------- Base model ----------
BASE_MODEL = os.environ.get("BASE_MODEL", "Qwen/Qwen2.5-7B-Instruct")

assert DATA_PATH.exists(), f"Dataset not found: {DATA_PATH}"

device = "cuda" if torch.cuda.is_available() else "cpu"
print("Device:", device)

# 4-bit config (only used if CUDA is available)
bnb_config = None
if device == "cuda":
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_use_double_quant=True,
        bnb_4bit_compute_dtype=torch.bfloat16
    )


Device: cuda


In [2]:
from peft import PeftModel

base = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    torch_dtype=torch.bfloat16 if device=="cuda" else torch.float32,
    device_map="auto" if device=="cuda" else None,
    quantization_config=bnb_config
)
base = PeftModel.from_pretrained(base, str(OUTPUT_DIR))


ValueError: Some modules are dispatched on the CPU or the disk. Make sure you have enough GPU RAM to fit the quantized model. If you want to dispatch the model on the CPU or the disk while keeping these modules in 32-bit, you need to set `llm_int8_enable_fp32_cpu_offload=True` and pass a custom `device_map` to `from_pretrained`. Check https://huggingface.co/docs/transformers/main/en/main_classes/quantization#offload-between-cpu-and-gpu for more details. 